In [1]:
using GAMS

In [2]:
using JuMP

In [3]:
using DataStructures

In [4]:
using DataFrames

In [5]:
using Surrogates

In [6]:
using Flux

In [7]:
using Statistics

In [8]:
using GLPK

# Novel mathematical formulation for the short-term scheduling of batch plants: Example 3

This Model presents an application of the novel mathematical formulation for the short-term scheduling of batch plants. The proposed formulation by Ierapetritou and Floudas is used on a literature example by Susarla et al. that involves 4 tasks and 4 units.

Formulation Source:
Ierapetritou and Floudas (1998) Effective Continuous-Time Formulation for Short-Term Scheduling. 1. Multipurpose Batch Processes. Independent Engineering Chemical Research 37 (4341-4359)

Example Source:
Susarla et al. (2009) A Novel Approach to Scheduling Multipurpose Batch Plants Using Unit-Slots: MILP Example 1. AIChe Wiley InterScience Process Systems Engineering (1862 - 1866).

Keywords:scheduling, multipurpose plants, batch processes, slot-based formulations, unit-slots, continuous-time

In [9]:
H = 8;
P = 100;

# SETS

Define the sets to be utilised in the MILP
Please task note that if a task can be performed in more than one unit then it is considered a different task for each unit it can be performed in.


In [10]:
I = ["t1","t2","t3","t4"];
J = ["j1","j2","j3","j4"];
N = range(1,P, step = 1);
S = ["s1","s2","s3","s4","s5","s6"];

Jt1 = [J[1]];
Jt2 = [J[2]];
Jt3 = [J[3]];
Jt4 = [J[4]];

Ij1 = [I[1]];
Ij2 = [I[2]];
Ij3 = [I[3]];
Ij4 = [I[4]];

Is1 = [I[1]];
Is2 = I[1:2];
Is3 = [I[1], I[3]];
Is4 = [I[2], I[4]];
Is5 = I[3:4];
Is6 = [I[4]];

Ip = I;
Jp = J;
Np = N;


# Parameters

The available information for the process is given below i.e. storage capacity, initial available inventory,
batch size and parameters alpha and beta which are the processing time parameters viz. constant and variable terms


In [11]:
vmin = OrderedDict(
   "t1" => 0,
   "t2" => 0,
   "t3" => 0,
   "t4" => 0
)

vmax = OrderedDict(
   "t1" => 40,
   "t2" => 20,
   "t3" => 5,
   "t4" => 40
)

#STin = [10000, 0, 0, 0]
STin = OrderedDict(
     "s1" => 10000,
     "s2" => 0,
     "s3" => 0,
     "s4" => 0,
     "s5" => 0,
     "s6" => 0

)

#STmax = [+inf, 200, 250, +inf ]
STmax = OrderedDict(
     "s1" => 10000,
     "s2" => 10,
     "s3" => 15,
     "s4" => 10,
     "s5" => 15,
     "s6" => 10000
)

#alpha = [1.333, 1.333, 1.000, 0.667, 0.667]
alpha = OrderedDict(
"t1" => 1.666,
"t2" => 2.333,
"t3" => 0.667,
"t4" => 2.667

)

#beta = [0.01333, 0.01333, 0.00500, 0.00445, 0.00445]
beta = OrderedDict(
"t1" => 0.03335,
"t2" => 0.08335,
"t3" => 0.01665,
"t4" => 0.02664
)

price = OrderedDict(
"s1" => 0,
"s2" => 0,
"s3" => 0,
"s4" => 0,
"s5" => 0,
"s6" => 10
)

demand = OrderedDict(
"s1" => 0,
"s2" => 0,
"s3" => 0,
"s4" => 0,
"s5" => 0,
"s6" => 0
)

#Table p(i,s) 'proportion of state s produced (p(i,s) > 0), consumed (p(i,s) < 0) by task i'
#            s1      s2       s3        s4       s5       s6
#  t1        -1      +0.5     +0.5
#  t2                -1                 +1
#  t3                         -1                 +1
#  t4                                   -0.5     -0.5     +1




rho_table = DataFrame([(I = "t1", s1 = -1, s2 = +0.5, s3 = +0.5, s4 = 0, s5 = 0, s6 = 0 ),
                   (I = "t2", s1 = 0, s2 = -1, s3 = 0, s4 = +1, s5 = 0, s6 = 0),
                   (I = "t3", s1 = 0, s2 = 0, s3 = -1, s4 = 0, s5 = +1, s6 = 0 ),
                   (I = "t4", s1 = 0, s2 = 0, s3 = 0, s4 = -0.5, s5 = -0.5, s6 = +1 ) ])

rho = OrderedDict( (r[:I],states) => r[Symbol(states)] for r in eachrow(rho_table), states in S);

# MODEL DEFINITION

In [12]:
example3_netprofit = Model(GLPK.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: GLPK

# VARIABLES

In [13]:
@variables example3_netprofit begin
    w[i in I, n in N], Bin
    y[j in J, n in N], Bin
    bm[i in I, j in J, n in N] >= 0
    d[s in S, n in N] >= 0
    ST[s in S, n in N] >= 0
    Ts[i in I, j in J, n in N] >= 0
    Tf[i in I, j in J, n in N] >= 0
end

# ALLOCATION CONSTRAINTS
The allocation constraints express that at each unit j and at an event point n only one of the tasks that can be performed
in this unit (i.e. i ∈ Ij ) should take place

In [14]:
@constraints example3_netprofit begin
    allocation1[j in J, n in N; j == J[1]],
      sum(w[i,n] for i in Ij1) == y[j,n]
    allocation2[j in J, n in N; j == J[2]],
      sum(w[i,n] for i in Ij2) == y[j,n]  
    allocation3[j in J, n in N; j == J[3]],
      sum(w[i,n] for i in Ij3) == y[j,n]
    allocation4[j in J, n in N; j == J[4]],
      sum(w[i,n] for i in Ij4) == y[j,n]  
end

# CAPACITY CONSTRAINTS
The capacity constraints specify the minimum or maximum amount of available material for a task to take place in a unit
Since vmin is zero for all of them, the minimum capacity constraint is not defined


In [15]:
@constraints example3_netprofit begin
    capacity1[i in I, j in Jt1, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity2[i in I, j in Jt2, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity3[i in I, j in Jt3, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    capacity4[i in I, j in Jt4, n in N],
      bm[i,j,n] <= vmax[i]*w[i,n]
    
end

# STORAGE CONSTRAINTS
This constraint specifies the maximum storage capacity of each material state

In [16]:
@constraints example3_netprofit begin
    storage[s in S, n in N],
      ST[s,n] <= STmax[s]
end

# MATERIAL BALANCE: INITIAL CONDITIONS
This first set constraints is specifying the initial amount of each material state at event point 1


In [17]:
@constraints example3_netprofit begin
    matbal1a[s in S, n in N; s == S[1] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is1 for j in Jt1) - d[s,n]
    matbal1b[s in S, n in N; s == S[2] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is2 for j in Jt2) - d[s,n]
    matbal1c[s in S, n in N; s == S[3] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is3 for j in Jt3) - d[s,n]
    matbal1d[s in S, n in N; s == S[4] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is4 for j in Jt4) - d[s,n]
    matbal1e[s in S, n in N; s == S[5] && n == N[1]],
      ST[s,n] == STin[s] + sum(rho[i,s]*bm[i,j,n] for i in Is5 for j in Jt4) - d[s,n]
    matbal1f[s in S, n in N; s == S[6] && n == N[1]],
      ST[s,n] == STin[s] - d[s,n]
    
end

# MATERIAL BALANCE: PROCESSED
The second set of material balance performs a mass balnce on each state i.e. the amount of material s consumed at event point n by task i in unit j and the amount of material s produced at event point n-1 by task i in unit j


In [18]:
@constraints example3_netprofit begin
    matbal2a[s in S, n in 2:length(N); s == S[1]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt1 for i in Is1) - d[s,n]
    matbal2b[s in S, n in 2:length(N); s == S[2]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt2 for i in Is2) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt1 for i in Is2 ) - d[s,n]
    matbal2c[s in S, n in 2:length(N); s == S[3]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt3 for i in Is3) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt1 for i in Is3 )  - d[s,n]
    matbal2d[s in S, n in 2:length(N); s == S[4]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt3 for i in Is4) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt2 for i in Is4 ) - d[s,n]
    matbal2e[s in S, n in 2:length(N); s == S[5]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt4 for i in Is5) + sum(rho[i,s]*bm[i,j,n-1] for j in Jt3 for i in Is5 )- d[s,n]
    matbal2f[s in S, n in 2:length(N); s == S[6]],
      ST[s,n] == ST[s,n-1] + sum(rho[i,s]*bm[i,j,n] for j in Jt4 for i in Is6) - d[s,n]
    
end

# DURATION CONSTRAINTS
These constraints don't only specify how long task i will take in unit j but also specifies the dependence of the duration
on the amount of material to be processed by task i in unit j.

In [19]:
@constraints example3_netprofit begin
    duration1[i in I, j in Jt1, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration2[i in I, j in Jt2, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration3[i in I, j in Jt3, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
    duration4[i in I, j in Jt4, n in N],
      Tf[i,j,n] == Ts[i,j,n] + alpha[i]*w[i,n] + beta[i]*bm[i,j,n]
end

# SEQUENCE CONSTRAINTS

# i. Same Task in Same Unit
The first set of sequence specify that the start of task i at event point n+1 should start after the end of event point n
for the same task performed in unit j.

In [20]:
@constraints example3_netprofit begin
    sequence1a[i in I, j in Jt1, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] -  H*(2 - w[i,n]- y[j,n])
    sequence1b[i in I, j in Jt2, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1c[i in I, j in Jt3, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])
    sequence1d[i in I, j in Jt4, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Tf[i,j,n] - H*(2 - w[i,n]- y[j,n])

    sequence2a[i in I, j in Jt1, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2b[i in I, j in Jt2, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2c[i in I, j in Jt3, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]
    sequence2d[i in I, j in Jt4, n in 1:length(N)-1],
      Ts[i,j,n+1] >= Ts[i,j,n]

    sequence3a[i in I, j in Jt1, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3b[i in I, j in Jt2, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3c[i in I, j in Jt3, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
    sequence3d[i in I, j in Jt4, n in 1:length(N)-1],
      Tf[i,j,n+1] >= Tf[i,j,n]
end

# ii. Different Task in Same Unit
The following constraints establishes the relationship between the starting time of task i at point n+1 and the end time of task i′ (ip) at event point n when different tasks take place in the same unit.

In [21]:
@constraints example3_netprofit begin
    sequence4a[i in Ij1, ip in Ij1, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4b[i in Ij2, ip in Ij2, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4c[i in Ij3, ip in Ij3, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
    sequence4d[i in Ij4, ip in Ij4, j in J, n in 1:length(N)-1; i != ip],
      Ts[i,j,n+1] >= Tf[ip,j,n] - H*(2 - w[ip,n] - y[j,n])
end

# ii. Different Task in Different Units
When different tasks i and i′ are performed in different units j and j′ but take place one after the other according to the production recipe.
These constraints specify the order in which then tasks in each unit should be performed i.e. task 1 then task 2 (task 3 can't be completed until both task 1 and task 2 are completed) then task 4.


In [22]:
@constraints example3_netprofit begin
    sequence5a[i in I, ip in I, j in Jt2, jp in J, n in 1:length(N)-1; i == I[2] && ip == I[1] && jp == J[1]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5b[i in I, ip in I, j in Jt3, jp in J, n in 1:length(N)-1; i == I[3] && ip == I[2] && jp == J[2]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5c[i in I, ip in I, j in Jt4, jp in J, n in 1:length(N)-1; i == I[4] && ip == I[3] && jp == J[3]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
    sequence5d[i in I, ip in I, j in Jt3, jp in J, n in 1:length(N)-1; i == I[3] && ip == I[4] && jp == J[4]],
      Ts[i,j,n+1] >= Tf[ip,jp,n] - H*(2 - w[ip,n] - y[jp,n])
end

# iv. Completion of previous tasks
A task i' (ip) performed in unit j cannot start until task i in unit j is completed


In [23]:
@constraints example3_netprofit begin
   sequence6a[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence6b[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence6c[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence6d[i in I, j in Jt1, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)

   sequence7a[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence7b[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence7c[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence7d[i in I, j in Jt2, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)

   sequence8a[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence8b[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence8c[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence8d[i in I, j in Jt3, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)

   sequence9a[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij1)
   sequence9b[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij2)
   sequence9c[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij3)
   sequence9d[i in I, j in Jt4, n in 1:length(N)-1],
     Ts[i,j,n+1] >= sum(Tf[ip,j,np] - Ts[ip,j,np] for np in N if np <= n for ip in Ij4)

end

# TIME HORIZON CONSTRAINTS
Specify that all tasks should task place within the time horizon


In [24]:
@constraints example3_netprofit begin
    timehorizon1[i in I, j in J, n in N],
      Ts[i,j,n] <= H
    timehorizon2[i in I, j in J, n in N],
      Tf[i,j,n] <= H
    
end

# OBJECTIVE FUNCTION

## Profit maximization

In [25]:
@objective example3_netprofit Max begin
    sum(price[s]*d[s,n] for n in N for s in S if s == S[6])
end

10 d[s6,1] + 10 d[s6,2] + 10 d[s6,3] + 10 d[s6,4] + 10 d[s6,5] + 10 d[s6,6] + 10 d[s6,7] + 10 d[s6,8] + 10 d[s6,9] + 10 d[s6,10] + 10 d[s6,11] + 10 d[s6,12] + 10 d[s6,13] + 10 d[s6,14] + 10 d[s6,15] + 10 d[s6,16] + 10 d[s6,17] + 10 d[s6,18] + 10 d[s6,19] + 10 d[s6,20] + 10 d[s6,21] + 10 d[s6,22] + 10 d[s6,23] + 10 d[s6,24] + 10 d[s6,25] + 10 d[s6,26] + 10 d[s6,27] + 10 d[s6,28] + 10 d[s6,29] + 10 d[s6,30] + 10 d[s6,31] + 10 d[s6,32] + 10 d[s6,33] + 10 d[s6,34] + 10 d[s6,35] + 10 d[s6,36] + 10 d[s6,37] + 10 d[s6,38] + 10 d[s6,39] + 10 d[s6,40] + 10 d[s6,41] + 10 d[s6,42] + 10 d[s6,43] + 10 d[s6,44] + 10 d[s6,45] + 10 d[s6,46] + 10 d[s6,47] + 10 d[s6,48] + 10 d[s6,49] + 10 d[s6,50] + 10 d[s6,51] + 10 d[s6,52] + 10 d[s6,53] + 10 d[s6,54] + 10 d[s6,55] + 10 d[s6,56] + 10 d[s6,57] + 10 d[s6,58] + 10 d[s6,59] + 10 d[s6,60] + 10 d[s6,61] + 10 d[s6,62] + 10 d[s6,63] + 10 d[s6,64] + 10 d[s6,65] + 10 d[s6,66] + 10 d[s6,67] + 10 d[s6,68] + 10 d[s6,69] + 10 d[s6,70] + 10 d[s6,71] + 10 d[s6,72] + 1

In [ ]:
optimize!(example3_netprofit);

# Optimal value and schedule

In [ ]:
println("Optimal Solution: ", JuMP.objective_value(example3_netprofit))
println("Optimal Schedule: ")
println(JuMP.value.(w))
println(JuMP.value.(bm))
println(JuMP.value.(ts))
println(JuMP.value.(tf))